# Attack image files and test prediction after attacking.


In [1]:
# from CNN_GTzan-2D-75w-LoadAndTest-110250

import gc
import numpy as np
import soundfile as sf
from numpy import *
import soundfile as sf
import keras
from keras import regularizers
from keras import backend
import os, sys
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten
from keras.callbacks import TensorBoard
from keras.utils import np_utils, to_categorical
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
import keras.initializers as init

from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks import BasicIterativeMethod
#from cleverhans.attacks import DeepFool
#from cleverhans.attacks import CarliniWagnerL2
from cleverhans.utils_keras import KerasModelWrapper

os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto( )
config.gpu_options.allow_growth = True
sess   = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)

Using TensorFlow backend.


In [2]:
# Controling Hyperparameters
nb_classes = 10
frame_size = 110250

In [3]:
#Indicate folds
train_fold = [2, 3]
test_fold = [1]

o_fold = str(test_fold[0])
f_s_folds = str(train_fold[0])+'-'+str(train_fold[1])

In [4]:
X_valid = np.load(str("folds_mf/2_GTzan_Xs_test_fold"+o_fold+"_110250_75_frozen.npy"))
#X_valid = np.load(str("folds_stft/2_GTzan_Xs_test_fold"+o_fold+"_110250_75_frozen_stft.npy"))

In [5]:
Y_valid = np.load( "folds_mf/2_GTzan_Ys_test_fold"+o_fold+"_110250_75_frozen.npy" )
t_valid = np.load( "folds_mf/2_GTzan_ts_test_fold"+o_fold+"_110250_75_frozen.npy" )
s_valid = np.load( "folds_mf/2_GTzan_ss_test_fold"+o_fold+"_110250_75_frozen.npy" )
#Y_valid = np.load( "folds_stft/2_GTzan_Ys_test_fold"+o_fold+"_110250_75_frozen_stft.npy" )
#t_valid = np.load( "folds_stft/2_GTzan_ts_test_fold"+o_fold+"_110250_75_frozen_stft.npy" )
#s_valid = np.load( "folds_stft/2_GTzan_ss_test_fold"+o_fold+"_110250_75_frozen_stft.npy" )

In [6]:
#Adapt 1D data to 2D CNN

X_valid = np.squeeze(X_valid)
X_valid = np.expand_dims(X_valid, axis = 3)

print(X_valid.shape, Y_valid.shape)

len_X_valid = X_valid.shape[0]
f = X_valid.shape[1]
g = X_valid.shape[2]

(7138, 64, 431, 1) (7138, 10)


In [7]:
def model_generator_GTzannet2D_1a():
    
    from keras.layers      import Input, Dense, Conv2D, AveragePooling1D, LeakyReLU, MaxPool2D, Flatten
    from keras.layers.core import Dropout
    from keras.models      import Model
    from keras             import initializers, optimizers, regularizers
    from keras.callbacks   import ModelCheckpoint
    from keras.utils       import multi_gpu_model
    
    from keras.layers.normalization import BatchNormalization
        
    import keras.initializers as init
    
    from kapre.utils          import Normalization2D
    from kapre.augmentation   import AdditiveNoise    
    
    sr = 22050
    
    inp   = Input(shape = (f, g, 1))    
    #----------------------
    conv1  = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(inp)
    norm1  = BatchNormalization()(conv1)
    #----------------------
    conv2  = Conv2D(filters = 32, kernel_size = (3, 3) )(norm1)
    act2   = LeakyReLU(alpha = 0.2)(conv2)
    pool2  = MaxPool2D(pool_size = 2, strides = 2)(act2)
    drop2  = Dropout(0.05)(pool2)
    #----------------------
    conv3  = Conv2D(filters = 64, kernel_size = (3, 3) )(drop2)
    act3   = LeakyReLU(alpha = 0.2)(conv3)
    #----------------------
    conv4  = Conv2D(filters = 64, kernel_size = (3, 3) )(act3)
    act4   = LeakyReLU(alpha = 0.2)(conv4)
    pool4  = MaxPool2D(pool_size = 4, strides = 2)(act4)
    #----------------------
    flat   = Flatten()(pool4)
    #----------------------    
    #dense3 = Dense(512, activation='relu', kernel_initializer = initializers.glorot_uniform( seed = 0) )(flat)
    dense3 = Dense(1024, activation='relu', kernel_initializer = initializers.glorot_uniform( seed = 0) )(flat)
    drop3  = Dropout(0.80)(dense3)    
    #----------------------
    dense4 = Dense(nb_classes, activation='softmax')(drop3)
    #----------------------

    model  = Model(inp, dense4)

    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizers.Adadelta( lr = 1.0, rho = 0.95, epsilon = 1e-08, decay = 0.0),
                  metrics = ['accuracy'] )
    
    model.summary()
    
    return model

In [8]:
# Re-generating the model 
model = model_generator_GTzannet2D_1a()
path = "weights/weights_3_GTzan_3f_fold"+f_s_folds+"_20p_110250_75_frozen.best.hdf5"
#path = "weights/weights_3_GTzan_3f_fold"+f_s_folds+"_20p_110250_75_frozen_stft.best.hdf5"
model.load_weights(path)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 431, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 429, 32)       320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 429, 32)       128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 427, 32)       9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 60, 427, 32)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 213, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 213, 32)       0         
__________

## If adversarial samples already generated, skip section below

In [9]:
# Initialize the Attack objects

wrap = KerasModelWrapper(model)
#attack = FastGradientMethod(wrap, sess=sess)
attack = BasicIterativeMethod(wrap, sess=sess)


In [10]:
# Create a big matrix to accomodate test files

adversarial_matrix = np.ones((len_X_valid, f, g), dtype=float)
print(adversarial_matrix.shape)

(7138, 64, 431)


In [11]:
for i in range(0, len_X_valid):
    # Generates the adversarial example
    adversarial_example = attack.generate_np(X_valid[i:i+1], eps=0.3, eps_iter=0.1, nb_iter=10)
    adversarial_matrix[i,:] = np.squeeze(adversarial_example)
    if (i%500 == 0.000):
        print("Frames attacked: " +str(i))
print("Adversarial matrix filled")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Frames attacked: 0
Frames attacked: 500
Frames attacked: 1000
Frames attacked: 1500
Frames attacked: 2000
Frames attacked: 2500
Frames attacked: 3000
Frames attacked: 3500
Frames attacked: 4000
Frames attacked: 4500
Frames attacked: 5000
Frames attacked: 5500
Frames attacked: 6000
Frames attacked: 6500
Frames attacked: 7000
Adversarial matrix filled


In [12]:
## Save data files (input arrays) to disk (.npy files)

np.save( "folds_mf/2_GTzan_Xs_BIM_fold"+o_fold+"_110250_75_frozen.npy", adversarial_matrix)
#np.save( "folds_stft/2_GTzan_Xs_BIM_fold"+o_fold+"_110250_75_frozen_stft.npy", adversarial_matrix)

## End of part to skip

In [13]:
#Load matrix with adversarial samples of test fold

adversarial_matrix = np.load("folds_mf/2_GTzan_Xs_BIM_fold"+o_fold+"_110250_75_frozen.npy")
#adversarial_matrix = np.load("folds_stft/2_GTzan_Xs_BIM_fold"+o_fold+"_110250_75_frozen_stft.npy")

In [14]:
# Zips t_valid, s_valid with value of their original index 

indexes = list(range(0,len(t_valid)))
sorted_zip = sorted(zip(t_valid, s_valid, indexes))
sorted_t_valid, sorted_s_valid, sorted_indexes = zip(*sorted_zip)

In [15]:

#Combining similar segments

big_matrix = [] # Stores list of windows for each track, with the track label

print("Testing on test set of fold number: " + str(test_fold))
print_batch = 5       # Will print a message every 5 loops

m = 0
acc = 0

h = int(s_valid.max()) # all frames available
beg = t_valid.min()    # smallest id value out of the track values
end = t_valid.max()    # highest id value out of the track values
similar_items_res_sum = list()
y_target = None

for i in range(0, len(sorted_t_valid)):
    
    if (beg != sorted_t_valid[i]):
        
        if (beg%print_batch == 0):
            print("Done with:", beg, "out of", end)
        
        beg = sorted_t_valid[i]
        big_matrix.append([similar_items_res_sum, y_target])
        similar_items_res_sum = list()
    
    j = sorted_indexes[i] # Index of the original unsorted list
    
    # Loads the adversarial example
    adversarial_example = adversarial_matrix[j:j+1]
    
    y_proba   = model.predict(np.expand_dims(adversarial_example, axis=3))
    y_target  = Y_valid[j:j+1].argmax(axis=-1)
    
    m += 1
    if (y_proba.argmax() == Y_valid[j:j+1].argmax(axis=-1)):
        acc += 1
    
    similar_items_res_sum.append(y_proba)
    
big_matrix.append([similar_items_res_sum, y_target]) #Append last list

len_big_matrix = len(big_matrix)

print("Done testing - Total samples:", len_big_matrix)

Testing on test set of fold number: [1]
Done with: 665 out of 1000
Done with: 670 out of 1000
Done with: 675 out of 1000
Done with: 680 out of 1000
Done with: 685 out of 1000
Done with: 690 out of 1000
Done with: 695 out of 1000
Done with: 700 out of 1000
Done with: 705 out of 1000
Done with: 710 out of 1000
Done with: 715 out of 1000
Done with: 720 out of 1000
Done with: 725 out of 1000
Done with: 730 out of 1000
Done with: 735 out of 1000
Done with: 740 out of 1000
Done with: 745 out of 1000
Done with: 750 out of 1000
Done with: 755 out of 1000
Done with: 760 out of 1000
Done with: 765 out of 1000
Done with: 770 out of 1000
Done with: 775 out of 1000
Done with: 780 out of 1000
Done with: 785 out of 1000
Done with: 790 out of 1000
Done with: 795 out of 1000
Done with: 800 out of 1000
Done with: 805 out of 1000
Done with: 810 out of 1000
Done with: 815 out of 1000
Done with: 820 out of 1000
Done with: 825 out of 1000
Done with: 830 out of 1000
Done with: 835 out of 1000
Done with: 840 

In [16]:
#Accuracy on the entire set
print (acc, m, acc/m)

936 7138 0.1311291678341272


In [17]:
#Insert the number of splits (Example: 5 splits for 21 frames -> 1, 6, 12, 18, 21)

splits = [1, 6, 12, 18, 21]
        
percentage_vot = []
percentage_sum  = []

for i in range (len(splits)):
    percentage_vot.append(0)
    percentage_sum.append(0)
i = 0
    
for obj_similiar_items_res_sum in big_matrix:

    similar_items_res_sum = obj_similiar_items_res_sum[0]
    y_target = obj_similiar_items_res_sum[1]
    
    beg = 0
    index = 0
    lens = len(similar_items_res_sum) #To access the list of similar objects
    candidates_vot = np.zeros(nb_classes)
    candidates_sum = np.ones(nb_classes)
    
    for split in splits:
        
        current_split = split
        end = min(lens, split)
            
        for n in range(beg, end):
            candidates_vot[np.asarray(similar_items_res_sum[n]).argmax()] += 1 #voting
            candidates_sum = candidates_sum + similar_items_res_sum[n] #sum
            
        if (np.argmax(candidates_vot) == y_target):
            percentage_vot[index] += 1
        if (np.argmax(candidates_sum) == y_target):
            percentage_sum[index] += 1
            
        candidates_sum = candidates_sum.copy()
        candidates_vot = candidates_vot.copy()
            
        beg = current_split
        index += 1
        
    i += 1
    if (i%print_batch == 0):
        print("Done with:", i, "out of", len_big_matrix)

Done with: 5 out of 340
Done with: 10 out of 340
Done with: 15 out of 340
Done with: 20 out of 340
Done with: 25 out of 340
Done with: 30 out of 340
Done with: 35 out of 340
Done with: 40 out of 340
Done with: 45 out of 340
Done with: 50 out of 340
Done with: 55 out of 340
Done with: 60 out of 340
Done with: 65 out of 340
Done with: 70 out of 340
Done with: 75 out of 340
Done with: 80 out of 340
Done with: 85 out of 340
Done with: 90 out of 340
Done with: 95 out of 340
Done with: 100 out of 340
Done with: 105 out of 340
Done with: 110 out of 340
Done with: 115 out of 340
Done with: 120 out of 340
Done with: 125 out of 340
Done with: 130 out of 340
Done with: 135 out of 340
Done with: 140 out of 340
Done with: 145 out of 340
Done with: 150 out of 340
Done with: 155 out of 340
Done with: 160 out of 340
Done with: 165 out of 340
Done with: 170 out of 340
Done with: 175 out of 340
Done with: 180 out of 340
Done with: 185 out of 340
Done with: 190 out of 340
Done with: 195 out of 340
Done w

In [18]:
###### VOTING RULE
i = 0
samples = len(big_matrix)
for split in percentage_vot:
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1

Acc: 12.647058823529411 Number of tracks tested: 340 Number of frames combined: 1
Acc: 10.0 Number of tracks tested: 340 Number of frames combined: 6
Acc: 9.411764705882353 Number of tracks tested: 340 Number of frames combined: 12
Acc: 10.294117647058822 Number of tracks tested: 340 Number of frames combined: 18
Acc: 9.705882352941178 Number of tracks tested: 340 Number of frames combined: 21


In [19]:
###### SUM RULE
i = 0
samples = len(big_matrix)
for split in percentage_sum:
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1

Acc: 12.647058823529411 Number of tracks tested: 340 Number of frames combined: 1
Acc: 10.882352941176471 Number of tracks tested: 340 Number of frames combined: 6
Acc: 8.529411764705882 Number of tracks tested: 340 Number of frames combined: 12
Acc: 10.294117647058822 Number of tracks tested: 340 Number of frames combined: 18
Acc: 10.294117647058822 Number of tracks tested: 340 Number of frames combined: 21
